This notebook contains the code tha
t uses spaceyMoji to tokenize our text. This gets around the nltk problem

In [1]:
import spacy
import numpy as np
from spacymoji import Emoji
import pandas as pd
import emoji
import nltk
from gensim.models import Word2Vec

In [30]:
annSchiz1 = pd.read_csv('data/baseline/dataOut/annSchiz1.csv', encoding='utf-8')
annSchiz2 = pd.read_csv('data/baseline/dataOut/annSchiz2.csv', encoding='utf-8')
nonAnnSchizFile = pd.read_csv('data/dataIn/schiz/nonAnnFinalSchiz.csv', encoding='utf-8')

In [31]:
nlp = spacy.load('en')
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)

In [32]:
def checkEmoji(sent, check=False):
    doc = nlp(sent)
    if doc._.has_emoji==check:
        sent = np.nan
    return sent

def getEmojiDf(df, check=False):
    dfNew = df.copy()
    dfNew['Tweet'] = dfNew['Tweet'].apply(lambda x: checkEmoji(x, check))
    dfNew = dfNew.dropna()
    return dfNew

def removeEmojis(sent):
    doc = nlp(sent)
    tokens = [token.text for token in doc if not token._.is_emoji]
    sent = ' '.join(tokens)
    return sent

## Baseline

In [5]:
emojiDf = getEmojiDf(annSchiz1)
emojiDf.to_csv('data/baseline/emoji/emSchiz1Em.csv')
emojiDf['Classification'].value_counts()

1    31
0    25
Name: Classification, dtype: int64

In [6]:
noEmojiDf = emojiDf.copy()
noEmojiDf['Tweet'] = noEmojiDf['Tweet'].apply(lambda x: removeEmojis(x))
noEmojiDf.to_csv('data/baseline/emoji/Schiz1Em.csv')

In [7]:
emojiDf = getEmojiDf(annSchiz2)
emojiDf.to_csv('data/baseline/emoji/emSchiz2Em.csv')
emojiDf['Classification'].value_counts()

1    15
0    13
Name: Classification, dtype: int64

In [8]:
noEmojiDf = emojiDf.copy()
noEmojiDf['Tweet'] = noEmojiDf['Tweet'].apply(lambda x: removeEmojis(x))
noEmojiDf.to_csv('data/baseline/emoji/Schiz2Em.csv')

## Embeddings

In [31]:
min_count = [2, 3, 4, 5]
window = [2, 3 , 4, 5]
size = [25, 50, 100, 200]
sample=6e-5
alpha=0.05
min_alpha=0.0007 
negative=20
sgTrain = 1
cbowTrain = 0
emArgs = [min_count[0], window[0], size[3], sample, alpha, min_alpha, negative]

In [6]:
def tokenizeTweet(sent):
    doc = nlp(sent)
    tokens= [t.text.strip() for t in doc]
    return tokens

In [25]:
nonAnnSchiz = nonAnnSchizFile[['Tweet','true_false']]
#emojiDf = getEmojiDf(nonAnnSchiz)
emojitokens = nonAnnSchiz['Tweet'].apply(lambda x: tokenizeTweet(x))
emojitokens.to_csv('data/dataOut/schiz/emoji/nonAnnFinalSchizEmoji.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [50]:
charTokens = nonAnnSchiz['Tweet'].apply(lambda x: list(x))
charTokens.to_csv('data/dataOut/schiz/emoji/nonAnnFinalSchizEmojiChar.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [41]:
def getEmoticonText(tokens):

    emojiLines = list(map(lambda x: any(t._.is_emoji for t in x), tokens))
    tokens = np.array(tokens)
    emojiLines = np.array(emojiLines)
  
    tokens = tokens[emojiLines]
 
    return tokens

In [25]:
emojiLst = []
y = emojiDf['Tweet'].tolist()
for t in y:
    doc = nlp(t)
    for token in doc:
        if token._.is_emoji:
            emojiLst.append(token.text)

In [ ]:
## Train and Test Data 

In [33]:
emojitokens = annSchiz2['Tweet'].apply(lambda x: tokenizeTweet(x))

In [71]:
emojiDf = getEmojiDf(annSchiz2)
len(emojiDf)

28

In [ ]:
z = emojiDf['Tweet'].apply(lambda x: nltk.word_tokenize(x))
x = z.tolist()

In [55]:
emojitokens.to_csv('data/dataOut/schiz/emoji/annSchiz2.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [56]:
emojitokens = annSchiz2['Tweet'].apply(lambda x: list(x))
emojitokens.to_csv('data/dataOut/schiz/emoji/annSchizChar2.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
